In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import cloudscraper
import csv
import numpy as np
import re

location_pattern = re.compile('.?\.?us.?\.?')

In [ ]:
scraper = cloudscraper.create_scraper(debug=True)
states = []

url = 'https://locations.jackinthebox.com/us'

scrape_data = scraper.get(url).text
soup = BeautifulSoup(scrape_data)

for row in soup.find_all('a', href=True):
    states.append(row.get('href'))

states = [x for x in states if location_pattern.match(x)]

In [ ]:
cities = []

for x in states:
    url = 'https://locations.jackinthebox.com'
    url += x

    scrape_data = scraper.get(url).text
    soup = BeautifulSoup(scrape_data)
    
    for row in soup.find_all('a', href=True):
        cities.append(row.get('href'))

cities = [x for x in cities if location_pattern.match(x)]

In [ ]:
locations = []

for x in cities:
    url = 'https://locations.jackinthebox.com'
    url += x

    scrape_data = scraper.get(url).text
    soup = BeautifulSoup(scrape_data)
    
    for row in soup.find_all('a', href=True):
        locations.append(row.get('href'))

locations = [x for x in locations if location_pattern.match(x)]

In [ ]:
locations_final_df = pd.DataFrame(set(locations) - set(cities) - set(states))
locations_final_df.rename(columns={0: 'raw_url'}, inplace=True)

locations_final_df[['abc', 'country', 'state', 'city', 'address']] = locations_final_df['raw_url'].str.split('/', expand=True)
locations_final_df.drop(columns='abc', inplace = True)

locations_final_df.to_csv('JITB_Locations_20250507.csv')